**Fraud detection ML**

In [1]:
import pandas as pd

dataset_path = 'card_transdata.csv'

df = pd.read_csv(dataset_path)

print(df.head())

   distance_from_home  distance_from_last_transaction  \
0           57.877857                        0.311140   
1           10.829943                        0.175592   
2            5.091079                        0.805153   
3            2.247564                        5.600044   
4           44.190936                        0.566486   

   ratio_to_median_purchase_price  repeat_retailer  used_chip  \
0                        1.945940              1.0        1.0   
1                        1.294219              1.0        0.0   
2                        0.427715              1.0        0.0   
3                        0.362663              1.0        1.0   
4                        2.222767              1.0        1.0   

   used_pin_number  online_order  fraud  
0              0.0           0.0    0.0  
1              0.0           0.0    0.0  
2              0.0           1.0    0.0  
3              0.0           1.0    0.0  
4              0.0           1.0    0.0  


In [ ]:
#Summary statistics
print(df.describe())

In [ ]:
#Data distribution
import matplotlib.pyplot as plt

plt.hist(df['distance_from_home'], bins=20)
plt.xlabel('Distance from Home')
plt.ylabel('Frequency')
plt.title('Distribution of Distance from Home')
plt.show()


In [ ]:
#Correlation analysis
correlation_matrix = df.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

In [ ]:
#Class distribution
fraud_distribution = df['fraud'].value_counts()
print(fraud_distribution)

In [ ]:
#Feature Analysis
import seaborn as sns

sns.boxplot(x='fraud', y='distance_from_home', data=df)
plt.xlabel('Fraud')
plt.ylabel('Distance from Home')
plt.title('Distance from Home vs. Fraud')
plt.show()
